In [54]:
import pandas as pd
import matplotlib.pyplot as plt

In [68]:
agg = pd.read_csv('agg.csv', low_memory=False)
agg = agg.drop(['index'], axis = 1)
agg = agg.drop(19, axis = 0)
agg = agg.reset_index()
agg = agg.drop(['index'], axis = 1)
weather = pd.read_csv('weather2.csv')
weather = weather.drop(['Unnamed: 0'], axis = 1)
park = pd.read_csv('park.csv')
park['YEAR'] = [i.replace('-01-01 00:00:00', '') for i in park['YEAR']]
park['YEAR'] = park['YEAR'].astype('int')
park = park[park['YEAR'] >= 2014]
park = park.reset_index()
park = park.drop(['index'], axis = 1)

In [70]:
abb = {'KAN' : 'KC', 'FLA' : 'MIA', 'CHW' : 'CWS', 'WAS' : 'WSH'}
for i in range(len(weather)):
    for word, initial in abb.items():
        weather.at[i, 'Team 1'] = weather.at[i, 'Team 1'].replace(word, initial)
        weather.at[i, 'Team 2'] = weather.at[i, 'Team 2'].replace(word, initial)

In [71]:
for i in range(len(weather)):
    weather.at[i, 'Year'] = weather.at[i, 'Date'].split('-')[0]
park['YEAR'] = park['YEAR'].astype('str')

In [72]:
weather.head()

,Date,Team 1,Team 2,Team 1 Score,Team 2 Score,Venue,Weather (Degrees),Weather Type,Wind (MPH),Wind Direction,Year
0,2014-03-22,ARI,LAD,1,3,Sydney Cricket Ground,75,Overcast,5,Varies,2014
1,2014-03-22,ARI,LAD,5,7,Sydney Cricket Ground,79,Partly Cloudy,5,Varies,2014
2,2014-03-30,LAD,SD,1,3,Petco Park,67,Clear,11,L To R,2014
3,2014-03-31,CHC,PIT,0,1,PNC Park,54,Sunny,2,Out To CF,2014
4,2014-03-31,KC,DET,3,4,Comerica Park,52,Sunny,4,In From CF,2014


In [73]:
park.head()

,YEAR,PARK NAME,RUNS,HR,H,2B,3B,BB,PARKID,LEAGUE,...,CF_wall_norm,CF_wall_stand,RCF_wall_norm,RCF_wall_stand,RF_wall_norm,RF_wall_stand,Area_norm,Area_stand,Back_norm,Back_stand
0,2014,Progressive Field,0.950,1.083,0.982,1.028,0.528,0.963,CLE08,AL,...,0.157895,-0.111632,0.2,-0.427418,0.5,0.49017,0.889262,-0.004813,1.0,2.275899
1,2015,Progressive Field,1.261,0.986,1.169,1.440,0.774,1.115,CLE08,AL,...,0.157895,-0.111632,0.2,-0.427418,0.5,0.49017,0.889262,-0.004813,1.0,2.275899
2,2016,Progressive Field,1.207,1.168,1.115,1.299,0.507,1.119,CLE08,AL,...,0.157895,-0.111632,0.2,-0.427418,0.5,0.49017,0.889262,-0.004813,1.0,2.275899
3,2017,Progressive Field,0.971,0.974,0.988,1.017,0.640,0.931,CLE08,AL,...,0.157895,-0.111632,0.2,-0.427418,0.5,0.49017,0.889262,-0.004813,1.0,2.275899
4,2018,Progressive Field,1.128,1.019,1.031,1.040,1.000,1.027,CLE08,AL,...,0.157895,-0.111632,0.2,-0.427418,0.5,0.49017,0.889262,-0.004813,1.0,2.275899


In [74]:
agg.head()

,Game_date,Games_played_to_date,BA,PA,Name,ISO,SLG,BABIP,BIP,OBP,...,pHitsByZone13,pHitsByZone14,pHitsByZone2,pHitsByZone3,pHitsByZone4,pHitsByZone5,pHitsByZone6,pHitsByZone7,pHitsByZone8,pHitsByZone9
0,2014-03-31,1,0.0,4,Travis d'Arnaud,0.0,0.0,0.0,0.33333,0.25000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014-04-02,2,0.0,7,Travis d'Arnaud,0.0,0.0,0.0,0.5,0.14286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014-04-03,3,0.0,11,Travis d'Arnaud,0.0,0.0,0.0,0.44444,0.18182,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014-04-04,4,0.0,14,Travis d'Arnaud,0.0,0.0,0.0,0.5,0.14286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2014-04-06,5,0.0,17,Travis d'Arnaud,0.0,0.0,0.0,0.6,0.11765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
mergeTemp = pd.merge(left = weather, right = park, how = "left", left_on = ['Year', 'Venue'], right_on = ['YEAR', 'PARK NAME'])
mergeTemp

,Date,Team 1,Team 2,Team 1 Score,Team 2 Score,Venue,Weather (Degrees),Weather Type,Wind (MPH),Wind Direction,...,CF_wall_norm,CF_wall_stand,RCF_wall_norm,RCF_wall_stand,RF_wall_norm,RF_wall_stand,Area_norm,Area_stand,Back_norm,Back_stand
0,2014-03-22,ARI,LAD,1,3,Sydney Cricket Ground,75,Overcast,5,Varies,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-03-22,ARI,LAD,5,7,Sydney Cricket Ground,79,Partly Cloudy,5,Varies,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-03-30,LAD,SD,1,3,Petco Park,67,Clear,11,L To R,...,0.052632,-0.643762,0.25,-0.227120,0.318182,-0.182397,0.923658,0.195112,0.692308,-1.459076
3,2014-03-31,CHC,PIT,0,1,PNC Park,54,Sunny,2,Out To CF,...,0.210526,0.154434,0.80,1.976155,0.818182,1.667163,0.932886,0.248750,0.784615,-0.338584
4,2014-03-31,KC,DET,3,4,Comerica Park,52,Sunny,4,In From CF,...,0.105263,-0.377697,0.45,0.574071,0.227273,-0.518681,0.973154,0.482809,0.800000,-0.151835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13618,2019-09-29,ATL,NYM,6,7,Citi Field,78,Sunny,13,In From LF,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
13619,2019-09-29,OAK,SEA,1,3,T-Mobile Park,53,Partly Cloudy,0,Calm,...,0.105263,-0.377697,0.15,-0.627716,0.227273,-0.518681,0.883389,-0.038946,0.846154,0.408411
13620,2019-09-29,DET,CWS,3,5,Guaranteed Rate Field,61,Overcast,11,L To R,...,0.105263,-0.377697,0.15,-0.627716,0.227273,-0.518681,0.883389,-0.038946,0.923077,1.342155
13621,2019-09-29,MIN,KC,4,5,Kauffman Stadium,86,Partly Cloudy,14,R To L,...,0.105263,-0.377697,0.15,-0.627716,0.227273,-0.518681,0.994128,0.604714,0.692308,-1.459076


In [76]:
mergeTemp2 = pd.merge(agg, mergeTemp, how = "inner", left_on = ["next_game_date", "HomeTeam", "AwayTeam"], right_on = ["Date", "Team 1", "Team 2"])
mergeTemp2

,Game_date,Games_played_to_date,BA,PA,Name,ISO,SLG,BABIP,BIP,OBP,...,CF_wall_norm,CF_wall_stand,RCF_wall_norm,RCF_wall_stand,RF_wall_norm,RF_wall_stand,Area_norm,Area_stand,Back_norm,Back_stand
0,2014-03-31,1,0.00000,4,Travis d'Arnaud,0.00000,0.00000,0.00000,0.33333,0.25000,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
1,2014-03-31,1,0.20000,5,Jayson Werth,0.00000,0.20000,0.33333,0.6,0.20000,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
2,2014-03-31,1,0.25000,5,Ian Desmond,0.00000,0.25000,0.25000,1,0.20000,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
3,2014-03-31,1,0.00000,2,Lucas Duda,0.00000,0.00000,0.00000,0.5,0.00000,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
4,2014-03-31,1,0.25000,5,Bryce Harper,0.00000,0.25000,0.50000,0.5,0.40000,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117952,2015-05-18,3,0.40000,10,Luis Sardinas,0.00000,0.40000,0.50000,0.8,0.40000,...,0.105263,-0.377697,0.45,0.574071,0.227273,-0.518681,0.973154,0.482809,0.800000,-0.151835
117953,2015-05-18,38,0.26056,159,J. D. Martinez,0.21860,0.47917,0.32955,0.66667,0.32704,...,0.105263,-0.377697,0.45,0.574071,0.227273,-0.518681,0.973154,0.482809,0.800000,-0.151835
117954,2015-05-18,37,0.29268,145,Adam Lind,0.23151,0.52419,0.33333,0.77419,0.38621,...,0.105263,-0.377697,0.45,0.574071,0.227273,-0.518681,0.973154,0.482809,0.800000,-0.151835
117955,2015-05-18,21,0.27381,88,Carlos Gomez,0.22619,0.50000,0.28788,0.81395,0.28409,...,0.105263,-0.377697,0.45,0.574071,0.227273,-0.518681,0.973154,0.482809,0.800000,-0.151835


In [77]:
mergeTemp3 = pd.merge(agg, mergeTemp, how = "inner", left_on = ["next_game_date", "HomeTeam", "AwayTeam"], right_on = ["Date", "Team 2", "Team 1"])
mergeTemp3

,Game_date,Games_played_to_date,BA,PA,Name,ISO,SLG,BABIP,BIP,OBP,...,CF_wall_norm,CF_wall_stand,RCF_wall_norm,RCF_wall_stand,RF_wall_norm,RF_wall_stand,Area_norm,Area_stand,Back_norm,Back_stand
0,2014-04-03,3,0.00000,11,Travis d'Arnaud,0.00000,0.00000,0.00000,0.44444,0.18182,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
1,2014-04-03,3,0.18182,13,Brandon Phillips,0.00000,0.18182,0.40000,0.45455,0.30769,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
2,2014-04-03,1,0.33333,4,Daniel Murphy,0.00000,0.33333,0.33333,1,0.50000,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
3,2014-04-03,3,0.00000,7,Lucas Duda,0.00000,0.00000,0.00000,0.5,0.14286,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
4,2014-04-02,2,0.28571,8,Ryan Ludwick,0.00000,0.28571,0.33333,0.85714,0.37500,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132183,2016-09-06,43,0.28346,143,Nick Franklin,0.18577,0.46923,0.34066,0.73846,0.33803,...,0.157895,-0.111632,0.30,-0.026822,0.363636,-0.014256,0.913591,0.136598,0.769231,-0.525332
132184,2016-09-06,90,0.26250,349,J. J. Hardy,0.14058,0.40308,0.28253,0.85231,0.29799,...,0.157895,-0.111632,0.30,-0.026822,0.363636,-0.014256,0.913591,0.136598,0.769231,-0.525332
132185,2016-09-03,39,0.19298,125,Caleb Joseph,0.01041,0.20339,0.23913,0.77966,0.23200,...,0.157895,-0.111632,0.30,-0.026822,0.363636,-0.014256,0.913591,0.136598,0.769231,-0.525332
132186,2016-09-06,92,0.25658,337,Pedro Alvarez,0.25490,0.51148,0.28713,0.72787,0.32047,...,0.157895,-0.111632,0.30,-0.026822,0.363636,-0.014256,0.913591,0.136598,0.769231,-0.525332


In [78]:
merge = mergeTemp2.append(mergeTemp3)
merge = merge[merge["HomeTeam"].notnull()]
merge = merge[merge["AwayTeam"].notnull()]
merge = merge[merge["next_game_date"].notnull()] 
merge

,Game_date,Games_played_to_date,BA,PA,Name,ISO,SLG,BABIP,BIP,OBP,...,CF_wall_norm,CF_wall_stand,RCF_wall_norm,RCF_wall_stand,RF_wall_norm,RF_wall_stand,Area_norm,Area_stand,Back_norm,Back_stand
0,2014-03-31,1,0.00000,4,Travis d'Arnaud,0.00000,0.00000,0.00000,0.33333,0.25000,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
1,2014-03-31,1,0.20000,5,Jayson Werth,0.00000,0.20000,0.33333,0.6,0.20000,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
2,2014-03-31,1,0.25000,5,Ian Desmond,0.00000,0.25000,0.25000,1,0.20000,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
3,2014-03-31,1,0.00000,2,Lucas Duda,0.00000,0.00000,0.00000,0.5,0.00000,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
4,2014-03-31,1,0.25000,5,Bryce Harper,0.00000,0.25000,0.50000,0.5,0.40000,...,0.263158,0.420499,0.65,1.375262,0.227273,-0.518681,0.927852,0.219493,0.707692,-1.272328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132183,2016-09-06,43,0.28346,143,Nick Franklin,0.18577,0.46923,0.34066,0.73846,0.33803,...,0.157895,-0.111632,0.30,-0.026822,0.363636,-0.014256,0.913591,0.136598,0.769231,-0.525332
132184,2016-09-06,90,0.26250,349,J. J. Hardy,0.14058,0.40308,0.28253,0.85231,0.29799,...,0.157895,-0.111632,0.30,-0.026822,0.363636,-0.014256,0.913591,0.136598,0.769231,-0.525332
132185,2016-09-03,39,0.19298,125,Caleb Joseph,0.01041,0.20339,0.23913,0.77966,0.23200,...,0.157895,-0.111632,0.30,-0.026822,0.363636,-0.014256,0.913591,0.136598,0.769231,-0.525332
132186,2016-09-06,92,0.25658,337,Pedro Alvarez,0.25490,0.51148,0.28713,0.72787,0.32047,...,0.157895,-0.111632,0.30,-0.026822,0.363636,-0.014256,0.913591,0.136598,0.769231,-0.525332


In [79]:
agg = agg[agg["HomeTeam"].notnull()]
agg = agg[agg["AwayTeam"].notnull()]
agg = agg[agg["next_game_date"].notnull()] 
agg.shape[0]

263794

In [80]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

x, y = pd.DataFrame(merge, columns = ['Games_played_to_date', 'BA', 'PA', 'ISO', 'SLG', 'BABIP', 'BIP', 'OBP', 'OPS', 'Weather (Degrees)', 'Wind (MPH)']), pd.DataFrame(merge, columns = ['next_game_HIT'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

clf = RandomForestClassifier()
clf.fit(x_train, y_train.values.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [81]:
predictions = clf.predict(x_test)
predictions = [float(i) for i in predictions]
comparison = pd.DataFrame({"Real" : y_test.iloc[:,0], "Predictions" : predictions})
comparison = comparison.reset_index()
comparison = comparison.drop(['index'], axis=1)
correct = 0
for i in range(len(comparison)):
    if comparison.at[i, 'Real'] == comparison.at[i, 'Predictions']:
        correct += 1
print (correct / len(comparison))

0.5915768854064642


In [82]:
false = comparison[comparison['Real'] != comparison['Predictions']]
false = false.reset_index()
false = false.drop(['index'], axis = 1)
falsePositive = 0
if len(false) == 0:
    print (falsePositive)
else:
    for i in range(len(false)):
        if false.at[i, 'Real'] == 0:
            falsePositive += 1
    print (falsePositive / len(false))

0.6653452748005677
